In [210]:
% pylab inline

%load_ext autoreload
%autoreload 2
import os
import time
import csv

import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

import indeed_scraping

DATA_DIR = os.path.join(os.getcwd(), 'data')
try:
    os.makedirs(DATA_DIR)
except FileExistsError:
    pass

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [337]:
df_top_cities

,CBSA,city,County,State,state_name
0,45060,Canastota,Madison,NY,New York
1,33100,Fort Lauderdale,Broward,FL,Florida
2,41620,Salt Lake City,Salt Lake,UT,Utah
3,33100,Doral,Miami-Dade,FL,Florida
4,28140,Kansas City,Wyandotte,KS,Kansas
5,36540,Omaha,Douglas,NE,Nebraska
6,17980,Columbus,Muscogee,GA,Georgia
7,38300,Pittsburgh,Allegheny,PA,Pennsylvania
8,31080,Santa Ana,Orange,CA,California
9,40900,Folsom,Sacramento,CA,California


In [333]:
os.listdir(DATA_DIR)

['cyber+security-senior_level-$120,000.tsv',
 'cyber+security-mid_level-$100,000.tsv',
 'cyber+security-mid_level-$110,000.tsv',
 'uscitiesv1.4.csv',
 'resumes-cyber+security-exp_61_120-bachelor.tsv',
 'resumes-cyber+security-exp_25_60-master.tsv',
 'resumes-cyber+security-exp_121+-bachelor.tsv',
 'cities_relative_cost.tsv',
 'resumes-cyber+security-exp_12_24-bachelor.tsv',
 '.DS_Store',
 'cyber+security-senior_level-$70,000.tsv',
 'cyber+security-entry_level-$90,000.tsv',
 'resumes-cyber+security-exp_121+-associate.tsv',
 'cyber+security-mid_level-$70,000.tsv',
 'resumes-cyber+security-exp_121+-doctor.tsv',
 'resumes-cyber+security-exp_25_60-diploma.tsv',
 'resumes-cyber+security-exp_12_24-associate.tsv',
 'resumes-cyber+security-exp_121+-master.tsv',
 'top_cities_cyril.csv',
 'cyber+security-entry_level-$120,000.tsv',
 'cities.tsv',
 'resumes-cyber+security-exp_12_24-doctor.tsv',
 'resumes-cyber+security-exp_12_24-master.tsv',
 'resumes-cyber+security-exp_61_120-diploma.tsv',
 'cyber

### areas

In [212]:
df_us_cities = pd.read_csv(os.path.join(DATA_DIR, 'uscitiesv1.4.csv'))
df_us_cities['county'] = df_us_cities['county_name']
df_us_cities = df_us_cities[['state_id', 'city', 'county']]

In [213]:
df_us_cities.head()

,state_id,city,county
0,WA,Prairie Ridge,Pierce
1,WA,Edison,Skagit
2,WA,Packwood,Lewis
3,WA,Wautauga Beach,Kitsap
4,WA,Harper,Kitsap


In [214]:
df_cbsa = pd.read_csv(os.path.join(DATA_DIR, 'cbsa-est2017-alldata.csv'), encoding = "latin1")

In [215]:
df_cbsa[['CBSA', 'NAME', 'STCOU']].columns

Index(['CBSA', 'NAME', 'STCOU'], dtype='object')

In [216]:
df_cbsa = df_cbsa[['CBSA', 'NAME', 'STCOU']]
df_cbsa['county'] = df_cbsa['NAME'].apply(lambda x: x.split(', ')[0].replace(' County', ''))
df_cbsa['state_id'] = df_cbsa['NAME'].apply(lambda x: x.split(', ')[1])

In [217]:
df_cbsa.head()

,CBSA,NAME,STCOU,county,state_id
0,24940,"Abbeville County, SC",45001.0,Abbeville,SC
1,10100,"Aberdeen, SD",NaN,Aberdeen,SD
2,10140,"Aberdeen, WA",NaN,Aberdeen,WA
3,10180,"Abilene, TX",NaN,Abilene,TX
4,29180,"Acadia Parish, LA",22001.0,Acadia Parish,LA


In [218]:
df_areas = df_cbsa.join(df_us_cities.set_index(['county', 'state_id']), on=['county', 'state_id'], how='inner')
df_areas = df_areas[['CBSA', 'county', 'state_id', 'city']]

In [219]:
df_areas.head()

,CBSA,county,state_id,city
0,24940,Abbeville,SC,Antreville
0,24940,Abbeville,SC,Donalds
0,24940,Abbeville,SC,Calhoun Falls
0,24940,Abbeville,SC,Lowndesville
0,24940,Abbeville,SC,Lake Secession


In [220]:
df_areas = df_areas.reset_index(drop=True)

### Get demand

In [221]:
df_demand = pd.read_csv(os.path.join(DATA_DIR, 'cyber+security_all_partitions.tsv'), sep='\t')
df_demand = df_demand[['company_name', 'location', 'partition', 'title']]
df_demand['location'] = df_demand['location'].str.strip()

In [222]:
len(df_demand)

23206

In [223]:
df_demand = df_demand[df_demand['location'].apply(lambda x: ', ' in x)]
df_demand['city'] = df_demand['location'].apply(lambda x: x.split(', ')[0])
df_demand['state_id'] = df_demand['location'].apply(lambda x: x.split(', ')[1])

In [224]:
df_demand.head()

,company_name,location,partition,title,city,state_id
0,Jackson-National-Life-Insurance-Company,"Lansing, MI","cyber+security-senior_level-$120,000","AVP, Cybersecurity Response",Lansing,MI
1,Occidental-Petroleum,"Houston, TX","cyber+security-senior_level-$120,000",IT Cyber Security Advisor,Houston,TX
2,\n Ingersoll Consulting Inc.,"Washington, DC","cyber+security-senior_level-$120,000",Cyber Security Engineer - Lead,Washington,DC
3,Saab,"Syracuse, NY","cyber+security-senior_level-$120,000",Senior Staff Systems Engineer; Saab Defense an...,Syracuse,NY
4,\n Executive Office of Energy and Environme...,"Boston, MA","cyber+security-senior_level-$120,000",Chief Information Security Officer,Boston,MA


In [225]:
df_demand_area = (
    df_areas
    .reset_index(drop=True)
    .join(
        df_demand.set_index(['city', 'state_id']), 
        on=['city', 'state_id'], 
        how='right')
    .reset_index(drop=True)
)

In [226]:
df_demand_area.head()

,CBSA,county,state_id,city,company_name,location,partition,title
0,14260.0,Ada,ID,Boise,Dxc-Technology,"Boise, ID","cyber+security-senior_level-$120,000",Segment Chief Information Security Officer- Pu...
1,14260.0,Ada,ID,Boise,General-Dynamics-Information-Technology,"Boise, ID","cyber+security-senior_level-$120,000",Cyber O&M Engineer
2,14260.0,Ada,ID,Boise,General-Dynamics-Information-Technology,"Boise, ID","cyber+security-senior_level-$120,000",Cyber O&M Engineer
3,14260.0,Ada,ID,Boise,Dxc-Technology,"Boise, ID","cyber+security-senior_level-$120,000",Segment Chief Information Security Officer- Pu...
4,14260.0,Ada,ID,Boise,General-Dynamics-Information-Technology,"Boise, ID","cyber+security-senior_level-$120,000",Cyber O&M Engineer


In [227]:
len(df_demand)

22543

In [287]:
df_demand_area['jobs'] = 1
df_demand_count = pd.DataFrame(df_demand_area.groupby('CBSA').count()['jobs'].reset_index())

### Supply

In [228]:
df_supply = pd.read_csv(os.path.join(DATA_DIR, 'resumes_cyber+security_all_partitions.tsv'), sep='\t')
df_supply = df_supply[df_supply['location'].apply(lambda x: isinstance(x, str) and ', ' in x)]
df_supply['city'] = df_supply['location'].apply(lambda x: x.split(', ')[0])
df_supply['state_id'] = df_supply['location'].apply(lambda x: x.split(', ')[1])
df_supply = df_supply[['experience', 'company', 'degree', 'partition', 'city', 'state_id']]

In [229]:
df_supply.head()

,experience,company,degree,partition,city,state_id
0,Computer Operator,First Citizens Bank,Bachelor of Science,resumes-cyber+security-exp_61_120-bachelor,Fuquay-Varina,NC
1,"Senior Analyst, Information Security",General Dynamics Information Technology,Bachelor of Science,resumes-cyber+security-exp_61_120-bachelor,Waynesville,OH
2,Recruiting Assistant,Teleperformance,Bachelors of Science in Computer Science,resumes-cyber+security-exp_61_120-bachelor,Shreveport,LA
3,Network Engineer,Naval Undersea Warfare Center,B.S.,resumes-cyber+security-exp_61_120-bachelor,Buford,GA
4,IT Assistant,Sky Zone Recreational Center,Bachelors,resumes-cyber+security-exp_61_120-bachelor,Chesterfield,MO


In [230]:
df_supply_area = (
    df_areas
    .reset_index(drop=True)
    .join(
        df_supply.set_index(['city', 'state_id']), 
        on=['city', 'state_id'], 
        how='right')
    .reset_index()
)

In [231]:
# df_supply_area['CBSA'] = df_supply_area['CBSA'].astype(int)

In [232]:
df_supply_area.head()

,index,CBSA,county,state_id,city,experience,company,degree,partition
0,7,14260.0,Ada,ID,Boise,Data Center Technician,Microsoft Global Foundation Services,BS,resumes-cyber+security-exp_61_120-bachelor
1,7,14260.0,Ada,ID,Boise,Data Center Technician,Microsoft Global Foundation Services,BS,resumes-cyber+security-exp_61_120-bachelor
2,7,14260.0,Ada,ID,Boise,Data Center Technician,Microsoft Global Foundation Services,BS,resumes-cyber+security-exp_61_120-bachelor
3,7,14260.0,Ada,ID,Boise,Data Center Technician,Microsoft Global Foundation Services,BS,resumes-cyber+security-exp_61_120-bachelor
4,7,14260.0,Ada,ID,Boise,Counselor,Va Department of Corrections,Bachelor of Science,resumes-cyber+security-exp_61_120-bachelor


In [286]:
df_supply_area['resumes'] = 1
df_supply_count = pd.DataFrame(df_supply_area.groupby('CBSA').count()[['resumes']].reset_index())

### Affortability

In [310]:
os.listdir(DATA_DIR)
df_relative_cost = pd.read_csv(os.path.join(DATA_DIR, 'cities_relative_cost.tsv'), sep='\t')
df_relative_cost['location'] = df_relative_cost['city']
df_relative_cost = df_relative_cost.drop(columns=['city'], axis=1)[['location', 'relative_cost']]
df_relative_cost = df_relative_cost.reset_index(drop=True)
df_relative_cost = df_relative_cost[df_relative_cost['location'].apply(lambda x: isinstance(x, str) and ', ' in x)]
df_relative_cost['city'] = df_relative_cost['location'].apply(lambda x: x.split(', ')[0])
df_relative_cost['state_id'] = df_relative_cost['location'].apply(lambda x: x.split(', ')[1])


In [311]:
df_relative_cost_area = (
    df_areas
    .reset_index(drop=True)
    .join(
        df_relative_cost.set_index(['city', 'state_id']), 
        on=['city', 'state_id'], 
        how='right')
    .reset_index()
)

In [313]:
df_relative_cost_area['affortability'] = 1 / df_relative_cost_area['relative_cost']
df_relative_cost_area_mean = df_relative_cost_area[['CBSA', 'affortability']].groupby('CBSA').mean()


In [314]:
df_relative_cost_area_mean.head()

,affortability
CBSA,
10180.0,2.180452
10580.0,1.869011
10740.0,2.179520
10900.0,2.033766
11100.0,2.308799


### Poseidon_score

In [317]:
df_relative_cost_area_mean.head()

,affortability
CBSA,
10180.0,2.180452
10580.0,1.869011
10740.0,2.179520
10900.0,2.033766
11100.0,2.308799


In [318]:
df_demand_supply = (
    df_demand_count
    .reset_index(drop=True)
    .join(
        df_supply_count.set_index('CBSA'), 
        how='inner', 
        on='CBSA'))


In [319]:
df_demand_supply.head()

,CBSA,jobs,resumes
0,10420.0,3,61
1,10500.0,1,1
2,10540.0,1,4
3,10580.0,2,125
4,10740.0,23,84


In [320]:
df_supply_demand_relative_cost = (
    df_demand_supply
    .join(
        df_relative_cost_area_mean, 
        how='inner', 
        on='CBSA'))

In [321]:
df_supply_demand_relative_cost.head()

,CBSA,jobs,resumes,affortability
3,10580.0,2,125,1.869011
4,10740.0,23,84,2.179520
5,10900.0,12,91,2.033766
7,11100.0,9,30,2.308799
8,11460.0,3,58,1.768038


In [323]:
df_supply_demand_relative_cost['mean_jobs'] = df_supply_demand_relative_cost['jobs'].mean()
df_supply_demand_relative_cost['mean_resumes'] = df_supply_demand_relative_cost['resumes'].mean()

In [341]:
df_supply_demand_relative_cost['poseidon_score'] = (
    (df_supply_demand_relative_cost['resumes'] + df_supply_demand_relative_cost['mean_resumes']) /
    (df_supply_demand_relative_cost['jobs'] + df_supply_demand_relative_cost['mean_jobs']) * 
    df_supply_demand_relative_cost['affortability']
)
df_supply_demand_relative_cost = df_supply_demand_relative_cost.sort_values(by='poseidon_score', ascending=False).reset_index(drop=True)

In [348]:
df_poseidon = df_supply_demand_relative_cost

In [349]:
df_poseidon.to_csv(os.path.join(DATA_DIR, 'posoidon_scores.tsv'), sep='\t')

In [350]:
df_poseidon[:10]

,CBSA,jobs,resumes,affortability,tot_jobs,tot_resumes,mean_jobs,mean_resumes,poseidon_score
0,41700.0,31,753,2.199351,10817,30856,94.06087,268.313043,17.961065
1,45300.0,41,593,1.844013,10817,30856,94.06087,268.313043,11.759678
2,36420.0,6,245,2.269764,10817,30856,94.06087,268.313043,11.643909
3,40140.0,9,326,1.955115,10817,30856,94.06087,268.313043,11.274407
4,19100.0,216,1444,1.947947,10817,30856,94.06087,268.313043,10.757549
5,41180.0,23,242,2.459756,10817,30856,94.06087,268.313043,10.723017
6,33100.0,38,550,1.602249,10817,30856,94.06087,268.313043,9.928310
7,17820.0,45,419,1.914641,10817,30856,94.06087,268.313043,9.463180
8,28140.0,10,187,2.073653,10817,30856,94.06087,268.313043,9.073163
9,12060.0,203,1117,1.922744,10817,30856,94.06087,268.313043,8.966520


In [351]:
df_poseidon[-10:]

,CBSA,jobs,resumes,affortability,tot_jobs,tot_resumes,mean_jobs,mean_resumes,poseidon_score
105,19380.0,121,99,2.248532,10817,30856,94.06087,268.313043,3.840378
106,42200.0,16,11,1.410927,10817,30856,94.06087,268.313043,3.580658
107,47900.0,2927,5714,1.568199,10817,30856,94.06087,268.313043,3.105352
108,35620.0,1249,2321,1.469075,10817,30856,94.06087,268.313043,2.832259
109,26620.0,532,282,2.259959,10817,30856,94.06087,268.313043,1.986524
110,41940.0,297,338,1.201934,10817,30856,94.06087,268.313043,1.863516
111,41860.0,417,484,1.244465,10817,30856,94.06087,268.313043,1.831929
112,42660.0,516,416,1.556154,10817,30856,94.06087,268.313043,1.745557
113,14460.0,554,468,1.413643,10817,30856,94.06087,268.313043,1.606151
114,29620.0,415,52,2.140669,10817,30856,94.06087,268.313043,1.346959


In [ ]:
os.path.join()

In [338]:
df_top_cities = pd.read_csv(os.path.join(DATA_DIR, 'top_cities_cyril.csv'))

In [354]:
df_top_cities

,CBSA,city,County,State,state_name
0,45060,Canastota,Madison,NY,New York
1,33100,Fort Lauderdale,Broward,FL,Florida
2,41620,Salt Lake City,Salt Lake,UT,Utah
3,33100,Doral,Miami-Dade,FL,Florida
4,28140,Kansas City,Wyandotte,KS,Kansas
5,36540,Omaha,Douglas,NE,Nebraska
6,17980,Columbus,Muscogee,GA,Georgia
7,38300,Pittsburgh,Allegheny,PA,Pennsylvania
8,31080,Santa Ana,Orange,CA,California
9,40900,Folsom,Sacramento,CA,California


In [353]:
df_poseidon.join(df_top_cities.set_index('CBSA'), on='CBSA', how='right').sort_values(by='poseidon_score', ascending=False)

,CBSA,jobs,resumes,affortability,tot_jobs,tot_resumes,mean_jobs,mean_resumes,poseidon_score,city,County,State,state_name
6,33100,38,550,1.602249,10817,30856,94.06087,268.313043,9.928310,Fort Lauderdale,Broward,FL,Florida
6,33100,38,550,1.602249,10817,30856,94.06087,268.313043,9.928310,Doral,Miami-Dade,FL,Florida
8,28140,10,187,2.073653,10817,30856,94.06087,268.313043,9.073163,Kansas City,Wyandotte,KS,Kansas
23,38300,41,337,1.734351,10817,30856,94.06087,268.313043,7.772978,Pittsburgh,Allegheny,PA,Pennsylvania
27,36540,18,133,2.113289,10817,30856,94.06087,268.313043,7.568121,Omaha,Douglas,NE,Nebraska
48,40900,15,155,1.712498,10817,30856,94.06087,268.313043,6.646955,Folsom,Sacramento,CA,California
69,41620,23,94,2.012675,10817,30856,94.06087,268.313043,6.229394,Salt Lake City,Salt Lake,UT,Utah
83,17980,7,37,1.878690,10817,30856,94.06087,268.313043,5.675673,Columbus,Muscogee,GA,Georgia
98,31080,258,836,1.557777,10817,30856,94.06087,268.313043,4.886296,Santa Ana,Orange,CA,California
104,45060,85,88,1.963573,10817,30856,94.06087,268.313043,3.907312,Canastota,Madison,NY,New York


In [355]:
SF_CBSA = ['10500','10580','10540','41860','34980','41940','41180','26900','18140','35620','24860','38300','25540']

In [357]:
df_poseidon[df_poseidon['CBSA'].isin(SF_CBSA)]

,CBSA,jobs,resumes,affortability,tot_jobs,tot_resumes,mean_jobs,mean_resumes,poseidon_score
5,41180.0,23,242,2.459756,10817,30856,94.06087,268.313043,10.723017
22,26900.0,25,232,1.854895,10817,30856,94.06087,268.313043,7.794569
23,38300.0,41,337,1.734351,10817,30856,94.06087,268.313043,7.772978
24,10580.0,2,125,1.869011,10817,30856,94.06087,268.313043,7.652507
64,34980.0,21,154,1.729716,10817,30856,94.06087,268.313043,6.348655
85,24860.0,8,59,1.769249,10817,30856,94.06087,268.313043,5.674048
90,25540.0,10,69,1.639269,10817,30856,94.06087,268.313043,5.313687
99,18140.0,134,238,1.953683,10817,30856,94.06087,268.313043,4.337329
108,35620.0,1249,2321,1.469075,10817,30856,94.06087,268.313043,2.832259
110,41940.0,297,338,1.201934,10817,30856,94.06087,268.313043,1.863516


In [358]:
df_poseidon[df_poseidon['CBSA'].isin(SF_CBSA)]['poseidon_score'].mean()

5.6494993458558405

In [329]:
.head(n=20)
45060
33100
41620
33100
28140
36540
17980
38300
31080
40900

,CBSA,jobs,resumes,affortability,tot_jobs,tot_resumes,mean_jobs,mean_resumes,poseidon_score
140,41700.0,31,753,2.199351,10817,30856,94.06087,268.313043,17.961065
162,45300.0,41,593,1.844013,10817,30856,94.06087,268.313043,11.759678
110,36420.0,6,245,2.269764,10817,30856,94.06087,268.313043,11.643909
131,40140.0,9,326,1.955115,10817,30856,94.06087,268.313043,11.274407
47,19100.0,216,1444,1.947947,10817,30856,94.06087,268.313043,10.757549
136,41180.0,23,242,2.459756,10817,30856,94.06087,268.313043,10.723017
102,33100.0,38,550,1.602249,10817,30856,94.06087,268.313043,9.928310
41,17820.0,45,419,1.914641,10817,30856,94.06087,268.313043,9.463180
84,28140.0,10,187,2.073653,10817,30856,94.06087,268.313043,9.073163
11,12060.0,203,1117,1.922744,10817,30856,94.06087,268.313043,8.966520


In [331]:
df_areas[df_areas['CBSA'] == 41700.0]

,CBSA,county,state_id,city
785,41700,Atascosa,TX,Christine
786,41700,Atascosa,TX,Jourdanton
787,41700,Atascosa,TX,Campbellton
788,41700,Atascosa,TX,Leming
789,41700,Atascosa,TX,Pleasanton
790,41700,Atascosa,TX,Coughran
791,41700,Atascosa,TX,Poteet
792,41700,Atascosa,TX,McCoy
793,41700,Atascosa,TX,Charlotte
794,41700,Atascosa,TX,Lytle


In [235]:
df_supply_area[df_supply_area['CBSA'] == 47900.0].head()

,index,CBSA,county,state_id,city,experience,company,degree,partition,resumes
1412,677,47900.0,Arlington,VA,Arlington,Verfiying Official/CAC Administrator,Mantech International Corporate,Bachelor of Science (on-going),resumes-cyber+security-exp_61_120-bachelor,1
1413,677,47900.0,Arlington,VA,Arlington,Verfiying Official/CAC Administrator,Mantech International Corporate,Bachelor of Science (on-going),resumes-cyber+security-exp_61_120-bachelor,1
1414,677,47900.0,Arlington,VA,Arlington,Verfiying Official/CAC Administrator,Mantech International Corporate,Bachelor of Science (on-going),resumes-cyber+security-exp_61_120-bachelor,1
1415,677,47900.0,Arlington,VA,Arlington,Verfiying Official/CAC Administrator,Mantech International Corporate,Bachelor of Science (on-going),resumes-cyber+security-exp_61_120-bachelor,1
1416,677,47900.0,Arlington,VA,Arlington,Verfiying Official/CAC Administrator,Mantech International Corporate,Bachelor of Science (on-going),resumes-cyber+security-exp_61_120-bachelor,1


In [236]:
supply_by_area.sort_values(by='count', ascending=False).head()

,CBSA,count
481,47900.0,5714
323,35620.0,2321
120,19100.0,1444
35,12580.0,1160
25,12060.0,1117


In [244]:
df_supply_demand = df_supply_demand.sort_values(by='resumes', ascending=False)
df_supply_demand.head()

,CBSA,jobs,resumes
170,47900.0,2927,5714
109,35620.0,1249,2321
47,19100.0,216,1444
16,12580.0,352,1160
11,12060.0,203,1117


In [246]:
df_supply_demand['competitive_rate'] = df_supply_demand['resumes'] / df_supply_demand['jobs']

In [255]:
df_supply_demand[
    (df_supply_demand['resumes'] > 100) &
    (df_supply_demand['jobs'] > 10)].sort_values(by='competitive_rate', ascending=True).head(n=20)

,CBSA,jobs,resumes,competitive_rate
140,41700.0,31,753,0.041169
102,33100.0,38,550,0.069091
162,45300.0,41,593,0.069140
74,25180.0,12,165,0.072727
33,16700.0,15,202,0.074257
136,41180.0,23,242,0.095041
133,40900.0,15,155,0.096774
41,17820.0,45,419,0.107399
81,26900.0,25,232,0.107759
34,16740.0,45,410,0.109756


#### Compare metropolitans

In [137]:
SF_CBSA = ['10500', '10580', '10540', '41860', '34980', '41940', '41180', '26900', '18140', '35620', '24860', '38300', '25540' ]

In [122]:
len(df_supply_area)

42738